# Começando do ZERO

In [2]:
import os
import pandas as pd
import sqlite3
import navegacao
import database

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


import pandas as pd
from openpyxl import Workbook
from openpyxl import load_workbook


### Funções Uteis

In [3]:
class Dataframe():
    def excluir_duplicatas_dataframe(self, dataframe, coluna):
        dataframe_sem_duplicatas = dataframe.drop_duplicates(subset=coluna, keep='first', inplace=False)
        print(f'Linhas duplicadas na coluna {coluna} excluídas.')
        return dataframe_sem_duplicatas

    def juntar_dataframes(self, df1, df2, coluna_chave: str=None):
        if coluna_chave!=None: df_junto = pd.merge(df1, df2, on=coluna_chave, how='inner')
        else: df_junto = pd.concat([df1, df2], ignore_index=True)
        print('DataFrames juntos:')
        print(df_junto)
        return df_junto
    
    def tratar_duplicadas_e_unir_dataframes(self, df, coluna_para_ver_duplicadas, coluna_para_avaliar, termo_para_avaliar):
        # Condição para considerar linhas duplicadas (exemplo: coluna 'A')
        condicao_duplicatas = df.duplicated(subset=coluna_para_ver_duplicadas)

        # Usando loc para selecionar as linhas duplicadas
        linhas_duplicadas = df.loc[condicao_duplicatas]

        linhas_duplicadas_com_condicao = linhas_duplicadas.loc[linhas_duplicadas[coluna_para_avaliar] == termo_para_avaliar]

        # Removendo as linhas duplicadas
        df_sem_duplicatas = df.drop(linhas_duplicadas.index)
        # df_sem_duplicatas = df.drop(df[df[coluna_para_ver_duplicadas].duplicated(keep='first')])
        
        df = self.juntar_dataframes(df_sem_duplicatas, linhas_duplicadas_com_condicao)
        return df


class Database(Dataframe):
    def __init__(self, db_name='minha_db.db'):
        self.conn = sqlite3.connect(db_name)

    def atualizando_tabelas_no_db(self, df, nome_da_tabela, coluna_a_unir):
        # Ler a tabela para um DataFrame
        df_lido = self.ler_tabela_para_dataframe(nome_da_tabela)

        # Juntar os DataFrames
        df_junto = self.juntar_dataframes(df_lido, df)

        # Excluir duplicatas no DataFrame resultante
        df_sem_duplicatas = self.excluir_duplicatas_dataframe(df_junto, coluna_a_unir)

        self.criar_tabela_do_dataframe(nome_da_tabela, df_sem_duplicatas)


    def criar_tabela_do_dataframe(self, tabela_nome, dataframe):
        dataframe.to_sql(tabela_nome, self.conn, index=False, if_exists='replace')
        print(f'Tabela {tabela_nome} adicionada ao banco de dados.')

    def ler_tabela_para_dataframe(self, tabela_nome):
        query = f'SELECT * FROM {tabela_nome}'
        df = pd.read_sql_query(query, self.conn)
        return df

    def excluir_tabela(self, tabela_nome):
        self.conn.execute(f'DROP TABLE IF EXISTS {tabela_nome}')
        print(f'Tabela {tabela_nome} excluída do banco de dados.')

    def fechar_conexao(self):
        self.conn.close()
        print('Conexão com o banco de dados fechada.')

def extraindo_empenhos(nave, ne, contrato):
    ## Extrair o extrato do EMPENHO
    nave.RELSALEMP(ne)
    nave.exportar_excel()
    relsalemp = os.path.join(os.path.expanduser("~"+os.sep+"Downloads"), "RELEXTRATOEMPENHO.xls")
    df = pd.read_excel(relsalemp)
    os.remove(relsalemp)

    colunas_excluir = [i for i in df.columns if not 'Unnamed' in i]
    linhas_excluir = [i for i in range(len(df)-2)]

    df = df.loc[linhas_excluir,colunas_excluir]
    df['CONTRATO'] = contrato

    return df

def procurar_texto(texto, busca):
    import re
    return re.search(busca, texto).group()




### Instancia a classe **navegação** e **Dataframe**

In [23]:
nave = navegacao.Navegacao()
Df = Dataframe()
Db = Database('Obras.db')

ano = 2021

### Busca os contratos na lista em excel

In [5]:
excel_contratos = r'C:\Users\luan.pinto\Desktop\Códigos\Projeto - Controle de obras\ARQUIVOS\CONTRATOS\contratos de obras - {ano}.xlsx'.format(ano=ano)


df_contratos = pd.read_excel(excel_contratos)
df_contratos = df_contratos[df_contratos['TIPO']=='OBRAS']

df_contratos.drop(['Descrição dos Contratos','DOCUMENTO', 'TIPO'], axis=1, inplace=True)

df_contratos['NOTAS DE EMPENHO'] = [i.split("'")[1] for i in df_contratos['NOTAS DE EMPENHO']]



### Busca as informações das NEs (Dotação Orcamentária) dos contratos pelo **RELSALEMP**

In [7]:
nave.entra_AFIM(ano)
lisne = []

df_relsalemp = pd.DataFrame()
for i,line in df_contratos.iterrows():
    ne = line['NOTAS DE EMPENHO']
    contrato = line['CONTRATO']

    lisne.extend(nave.LISNE(ne, detalhar_ne=True))
    lisne[-1].append(contrato)

    df_relsalemp = Df.juntar_dataframes(df_relsalemp,extraindo_empenhos(nave, ne, contrato))




=============== Executando entra_AFIM 2021! ===============
Esperando a página carregar por 10 segundos... 
Pagina carregada em 0.004 segundos!
0 https://afim2.manaus.am.gov.br/AfimPRD2021/logon.text
=============== Finalizando entra_AFIM 2021! ===============


=============== Executando LISNE! ===============
Esperando a página carregar por 10 segundos... 
Pagina carregada em 0.009 segundos!
0 https://afim2.manaus.am.gov.br/AfimPRD2021/Lisne.do
>>> Elemento "" encontrado em 0.019s!
>>> Elemento "×" encontrado em 0.01s!
Esperando a página carregar por 10 segundos... 
Pagina carregada em 0.005 segundos!
O termo "Relsalemp.do" não foi encontrado em nenhuma página!
Esperando a página carregar por 10 segundos... 
Pagina carregada em 0.004 segundos!
0 https://afim2.manaus.am.gov.br/AfimPRD2021/Relsalemp.do

=============== Executando exportar_excel! ===============
Esperando a página carregar por 10 segundos... 
Pagina carregada em 0.004 segundos!
1 http://mocatu.manaus.am.gov.br:8080/BOE

In [8]:
df_lisne = pd.DataFrame(lisne, columns=['Data de Emissão', 'Data de Lançamento', 'Número do Empenho', 'Credor', 'Evento', 'Unidade Orçamentária', 'Fonte de Recurso', 'Natureza da Despesa', 'Empenho Original', 'Processo', 'Valor', 'Descrição', 'Contrato'])



### Criando as tabelas para os ***Contratos***, as ***NEs*** e os ***Extratos dos empenhos*** no .db

In [9]:
Db.criar_tabela_do_dataframe('Contratos', df_contratos)
print('='*10, 'Contratos', '='*10)
display(df_contratos)

Db.criar_tabela_do_dataframe('LISNE', df_lisne)
print('='*10, 'LISNE', '='*10)
display(df_lisne)

Db.criar_tabela_do_dataframe('Relsalemp', df_relsalemp)
print('='*10, 'RELSALEMP', '='*10)
display(df_relsalemp)



Tabela Contratos adicionada ao banco de dados.
========== Contratos ==========


,CONTRATO,DATA DO CONTRATO,EMPRESA,OBJETO,VALOR,NOTAS DE EMPENHO
17,18,09.06.2021,RDS ENGENHARIA DE OBRAS E EMPREENDIMENTOS IMOB...,Obra de Construção de Talude – Rua Bela Vista ...,"271.269,96",0542
19,20,01.07.2021,CONSTRUTORA SOMA LTDA,"“Execução de Obra de Revitalização de Erosão, ...","4.465.537,18",0734
21,22,17.08.2021,TECNOARTE DA AMAZÔNIA CONSTRUÇÃO CIVIL LTDA,Obra e Serviços de Engenharia para a Construçã...,"347.643,14",00756
22,23,03.09.2021,AMAZONCRETO CONSTRUÇÕES EIRELI,Obra e Serviços de Engenharia para a Construçã...,"1.733.790,85",00809
24,25,26.10.2021,EDS CONSTRUÇÕES E SERVIÇOS LTDA,Contratação de Empresa para Prestação de Servi...,"3.488.673,60",01192
35,36,01.12.2021,AMAZON CRETO CONSTRUÇÕES EIRELI,Contratação de Empresa Especializada em Obras ...,"711.963,26",01518
36,37,20.12.2021,INTELLI PROJETOS E CONSTRUÇÕES LTDA,Contratação de Empresa Especializada em Obras ...,"125.572,08",01470
37,38,22.12.2021,VR CONSTRUÇÕES EIRELI,Contratação de Empresa Especializada em Obras ...,"3.056.166,20",01517
38,39,22.12.2021,SYRIA ENGENHARIA E CONSTRUÇÃO EIRELI,Contratação de Empresa Especializada em Obras ...,"184.935,72",01471
39,40,29.12.2021,DR7 SERVIÇO DE OBRAS DE ALVENARIA LTDA - ME,Contratação de Empresa Especializada em Obras ...,"320.000,00",01579


Tabela LISNE adicionada ao banco de dados.
========== LISNE ==========


,Data de Emissão,Data de Lançamento,Número do Empenho,Credor,Evento,Unidade Orçamentária,Fonte de Recurso,Natureza da Despesa,Empenho Original,Processo,Valor,Descrição,Contrato
0,09/06/2021,10/06/2021,2021NE00542,079037.640.001-36-RDS ENGENHARIA DE OBRAS E EM...,400091 - NCASP - Empenho de despesa,27101 - Secretaria Municipal de Infraestrutura,01000000 - Recursos Ordinários,44905117 - Obras de Infra-estrutura,,2021/20000/20006/0/000474,"271.269,96",VALOR CORRESPONDENTE A OBRA DE CONSTRUÇÃO DE T...,18
1,01/07/2021,26/07/2021,2021NE00734,010887.130.001-11-CONSTRUTORA SOMA LTDA,400091 - NCASP - Empenho de despesa,56701 - Fundo Municipal de Desenvolvimento Urbano,06100355 - Recursos do PROMINF / MANAUS / FINI...,44905117 - Obras de Infra-estrutura,,2021/20000/20031/0/000942,"4.465.537,18",OBRA EMERGENCIAL DE CONTENÇÃO DE EROSÃO NO CON...,20
2,04/08/2021,05/08/2021,2021NE00756,222368.400.001-90-TECNOARTE DA AMAZONIA CONSTR...,400091 - NCASP - Empenho de despesa,56701 - Fundo Municipal de Desenvolvimento Urbano,06100354 - Recursos do PROMINF / MANAUS / FINISA,44905117 - Obras de Infra-estrutura,,2021/20000/20031/0/000947,"347.643,14",VALOR CORRESPONDENTE A OBRA DE CONSTRUÇÃO DE G...,22
3,20/08/2021,20/08/2021,2021NE00809,073557.250.001-41-AMAZONCRETO CONSTRUÇÕES EIRELI,400091 - NCASP - Empenho de despesa,56701 - Fundo Municipal de Desenvolvimento Urbano,06100358 - REQUALIFICA III/MANAUS - Exercícios...,44905117 - Obras de Infra-estrutura,,2021/20000/20031/0/000967,"1.733.790,85",OBRA DE CONSTRUÇÃO DE CONTENÇÃO DE TALUDE E PR...,23
4,26/10/2021,27/10/2021,2021NE01192,055349.270.001-25-EDS CONSTUÇÕES E SERVIÇOS LTDA,400091 - NCASP - Empenho de despesa,27101 - Secretaria Municipal de Infraestrutura,01000000 - Recursos Ordinários,44905117 - Obras de Infra-estrutura,,2021/20000/20022/0/000952,"290.722,80",VALOR CORRESPONDENTE A CONTRATAÇÃO DE EMPRESA ...,25
5,01/12/2021,14/12/2021,2021NE01518,073557.250.001-41-AMAZONCRETO CONSTRUÇÕES EIRELI,400091 - NCASP - Empenho de despesa,56701 - Fundo Municipal de Desenvolvimento Urbano,06100354 - Recursos do PROMINF / MANAUS / FINISA,44905117 - Obras de Infra-estrutura,,2021/20000/20031/0/001210,"100.000,00",VALOR CORRESPONDENTE A OBRA DE REFORMA DA PONT...,36
6,13/12/2021,13/12/2021,2021NE01470,027422.840.001-17-INTELLI PROJETOS E CONSTRUÇÕ...,400091 - NCASP - Empenho de despesa,27101 - Secretaria Municipal de Infraestrutura,01000005 - Recursos Ordinários destinados às E...,44905196 - Poços Comunitários,,2021/20000/20031/0/001242,"125.572,08","Emenda Parlamentar ID Contrato: 2261, Emenda: ...",37
7,14/12/2021,14/12/2021,2021NE01517,132456.810.001-08-V R CONSTRUÇÕES EIRELI,400091 - NCASP - Empenho de despesa,56701 - Fundo Municipal de Desenvolvimento Urbano,06100358 - REQUALIFICA III/MANAUS - Exercícios...,44905117 - Obras de Infra-estrutura,,2021/20000/20031/0/001208,"100.000,00",CONSTRUÇÃO DE TALUDE E PRAÇA - RUA FÉLIX E RUA...,38
8,14/12/2021,14/12/2021,2021NE01471,844622.090.001-67-SYRIA ENGENHARIA E CONSTRUÇÃ...,400091 - NCASP - Empenho de despesa,56701 - Fundo Municipal de Desenvolvimento Urbano,06100354 - Recursos do PROMINF / MANAUS / FINISA,"44905193 - Reformas, Benfeitorias Ou Melhoria",,2021/20000/20031/0/001211,"184.935,72",Obra de recuperação estrutural da Ponte do Con...,39
9,23/12/2021,23/12/2021,2021NE01579,013355.160.001-50-DR7 SERVIÇO DE OBRAS DE ALVE...,400091 - NCASP - Empenho de despesa,27101 - Secretaria Municipal de Infraestrutura,01000005 - Recursos Ordinários destinados às E...,"44905193 - Reformas, Benfeitorias Ou Melhoria",,2021/20000/20053/0/001277,"320.000,00","Emenda Parlamentar ID Contrato: 2281, Emenda: ...",40


Tabela Relsalemp adicionada ao banco de dados.
========== RELSALEMP ==========


,Evento,Documento,Data,Cod.Evento,Valor,CONTRATO
0,Empenho,2021NE00542,09/06/2021,400091.0,271269.96,18
1,Em Liquidação,2021NL02390,25/08/2021,500001.0,164458.28,18
2,Liquidação,2021NL02465,02/09/2021,510013.0,164458.28,18
3,Pagamento por NL,2021NL02465,02/09/2021,520224.0,2466.87,18
4,Pagamento por NL,2021NL02465,02/09/2021,520504.0,822.29,18
...,...,...,...,...,...,...
81,Empenho,2021NE01517,14/12/2021,400091.0,100000,38
82,Anulação,2021NE01580,23/12/2021,400093.0,100000,38
83,Empenho,2021NE01471,14/12/2021,400091.0,184935.72,39
84,Anulação,2021NE01582,23/12/2021,400093.0,184935.72,39


### Fazendo uma lista com os empenhos dos anos seguintes

In [10]:
# df_lisne = Db.ler_tabela_para_dataframe('LISNE')
# df_lisne

In [12]:
# df_contratos_lisne = Db.ler_tabela_para_dataframe('Contratos_LISNE')

nave._vars['lista_nes'] = []

for a in range(ano, 2024):
    nave.entra_AFIM(a)
    for e in range(len(df_lisne)):
        credor = procurar_texto(df_lisne['Credor'][e], r'.+-\d+').replace('.','').replace('-','')
        print(f'= = = = = = Executando para "{df_lisne["Credor"][e]}" \n- len(nave._vars["lista_nes"]) = {len(nave._vars["lista_nes"])}! = = = = = =')
        detalhes_ne = nave.LISNE(favorecido=credor, detalhar_ne=True)
        if type(detalhes_ne)==list: nave._vars['lista_nes'].extend(detalhes_ne)
        else: print(detalhes_ne)
    print()

colunas=['Data de Emissão', 'Data de Lançamento', 'Número do Empenho', 'Credor', 'Evento', 'Unidade Orçamentária', 'Fonte de Recurso', 'Natureza da Despesa', 'Empenho Original', 'Processo', 'Valor', 'Descrição']

Db.criar_tabela_do_dataframe('Lista_de_NEs', pd.DataFrame(nave._vars['lista_nes'], columns=colunas))




=============== Executando entra_AFIM 2021! ===============
Esperando a página carregar por 10 segundos... 
Pagina carregada em 1.918 segundos!
0 https://afim2.manaus.am.gov.br/AfimPRD2021/logon.text
=============== Finalizando entra_AFIM 2021! ===============

= = = = = = Executando para "079037.640.001-36-RDS ENGENHARIA DE OBRAS E EMPREENDIMENTOS IMOBILIARIOS EIRELI" 
- len(nave._vars["lista_nes"]) = 0! = = = = = =

=============== Executando LISNE! ===============
Esperando a página carregar por 10 segundos... 
Pagina carregada em 0.198 segundos!
0 https://afim2.manaus.am.gov.br/AfimPRD2021/Lisne.do
>>> Elemento "270101 - Secretaria " encontrado em 0.008s!
>>> Elementos "#tableNotasTab1 > tbody > tr td.sorting_1 button" encontrados em 0.033s! (len(2))
>>> Elemento "×" encontrado em 0.013s!
>>> Elemento "×" encontrado em 0.003s!
= = = = = = Executando para "010887.130.001-11-CONSTRUTORA SOMA LTDA" 
- len(nave._vars["lista_nes"]) = 2! = = = = = =

=============== Executando LISNE! ==

### Criando um Dataframe do LISOB

In [29]:
df_contratos_lisne = Db.ler_tabela_para_dataframe('LISNE')
nave._vars['lista_obs'] = []

for a in range(ano, 2024):
    nave.entra_AFIM(a)
    for e in range(len(df_contratos_lisne)):
        print(f'= = = = = = Executando para "{df_contratos_lisne["Credor"][e]}" - len(nave._vars["lista_obs"]) = {len(nave._vars["lista_obs"])}! = = = = = =')
        credor = procurar_texto(df_contratos_lisne['Credor'][e], r'.+-\d+').replace('.','').replace('-','')
        nave.LISOB(credor)
        nave.LISOBGrid_detalha_ob()
    print()



=============== Executando entra_AFIM 2021! ===============
Esperando a página carregar por 10 segundos... 
Pagina carregada em 0.018 segundos!
0 https://afim2.manaus.am.gov.br/AfimPRD2021/logon.text
=============== Finalizando entra_AFIM 2021! ===============

= = = = = = Executando para "079037.640.001-36-RDS ENGENHARIA DE OBRAS E EMPREENDIMENTOS IMOBILIARIOS EIRELI" - len(nave._vars["lista_obs"]) = 0! = = = = = =

=============== Executando LISOB! (07903764000136) ===============
Esperando a página carregar por 10 segundos... 
Pagina carregada em 0.0 segundos!
0 https://afim2.manaus.am.gov.br/AfimPRD2021/Lisob.do
>>> Elemento "" encontrado em 0.0s!

=============== Finalizando LISOB! ===============
Esperando a página carregar por 10 segundos... 
Pagina carregada em 0.007 segundos!
1 https://afim2.manaus.am.gov.br/AfimPRD2021/LisObGrid.do
Esperando a página carregar por 10 segundos... 
Pagina carregada em 0.017 segundos!
2 https://afim2.manaus.am.gov.br/AfimPRD2021/SubDetaOB.do?fil

In [ ]:
df_obs = pd.DataFrame(nave._vars["lista_obs"], columns=['Data de Emissão','Data de Lançamento','Nº OB','UG Fin.','UG Fav.','Credor','Processo','Valor','Descrição','NE de Controle','NL de Controle'])

#16m

### Cálculo do Saldo dos Contratos

In [ ]:
df_relsalemp = Db.ler_tabela_para_dataframe('Relsalemp')
df_contratos_lisne = Db.ler_tabela_para_dataframe('Contratos_LISNE')


In [ ]:
saldos = {}
for index, linha in df_contratos_lisne.iterrows(): 
    print(linha['Número'], '* * *',linha['Favorecido'])
    # display(df_obs[df_obs['NE de Controle'].str.contains(linha['Número'])])
    # print(linha['Valor'])
    saldos[linha['Favorecido']] = 0

    aux = df_obs[df_obs['NE de Controle'].str.contains(linha['Número'])]
    for l in aux['Valor']: saldos[linha['Favorecido']] += float(l.replace('.', '').replace(',','.'))
    print(saldos)
    display(aux)




In [ ]:
nave.navegador.quit()

= = = = = = = = = = = = = = = = = = FIM = = = = = = = = = = = = = = = = = =

# COdigo complementar

In [ ]:
import gerar_relatorio as gerar
import os

gerar.gerar(os.getcwd())


In [ ]:
import sqlite3
import pandas as pd

caminho_obras_db = os.path.join(os.getcwd(), 'DATABASE', 'OBRAS.db')
conn = sqlite3.connect(caminho_obras_db)
df_relsalemp = pd.read_sql_query("SELECT * FROM Relsalemp_completo", conn)
df_contratos = pd.read_sql_query("SELECT * FROM Contratos", conn)

matriz = gerar.create_table_from_dataframe(df_relsalemp,'018/2021')

df = pd.DataFrame(matriz[1:], columns=matriz[0])

df
for i in matriz: print(i)

In [ ]:
dataframe = df_relsalemp
contrato = '018/2021'

# if '/' in contrato: contrato = contrato.split('/')[0]
# Exclua as linhas onde 'NL' aparece na coluna 'Coluna1'
dataframe = dataframe[~dataframe['Documento'].str.contains('NL')]

# O operador ~ nega o resultado do .str.contains(), mantendo as linhas que não contêm 'NL'.
dataframe = dataframe[dataframe['CONTRATO_N°'] == contrato]

valores = []
for index, row in dataframe.iterrows():
    print(index, row['Valor'])
    if 'Anulação' in row['Evento'] or 'Pagamento' in row['Evento']:
        valor = row['Valor']
        if type(valor) is str:
            if ',' in valor: 
                valores.append(float(valor.replace('.', '').replace(',', '.'))*-1)
            elif '.' in valor: 
                dataframe.loc[index, 'Valor'] = '{:,.2f}'.format(float(valor)).replace(',', '_').replace('.', ',').replace('_', '.')
                valores.append(float(valor)*-1)
    else: valores.append(float(row['Valor']))
    print(type(row['Valor']))
    print(valores)

display(dataframe)


In [ ]:
import sqlite3

# Conecta-se ao banco de dados
conn = sqlite3.connect(caminho_obras_db)
cursor = conn.cursor()

# Define as condições para exclusão
documento_condicao = '2021OB3414'
evento_condicao = 'Pagamento por NL'

# Executa a instrução SQL para excluir linhas com as condições especificadas
cursor.execute('DELETE FROM Relsalemp_completo_copy WHERE Documento = ? AND Evento = ?', (documento_condicao, evento_condicao))

# Commit para salvar as alterações
conn.commit()
# Fecha a conexão
conn.close()


In [ ]:

dataframe['Valor'] = valores
dataframe = dataframe[['Evento', 'Documento', 'Data', 'PROGRAMA_DE_TRABALHO', 'FONTE_DE_RECURSO', 'NATUREZA_DESPESA', 'CONTA_CONTÁBIL', 'Valor']]
# dataframe = dataframe.fillna('')

table_data = dataframe.values.tolist()
for l,linha in enumerate(table_data): 
    for c,coluna in enumerate(linha): 
        # Convertendo a informação para apenas valores numéricos
        if type(coluna)==str and ' - 'in coluna: 
            table_data[l][c] = coluna.split(' - ')[0]
        # Formate o número para o formato desejado
        if type(coluna)==float:
            table_data[l][c] = '{:,.2f}'.format(coluna).replace(',', '_').replace('.', ',').replace('_', '.')

# for i in table_data: 
table_data.insert(0, ['TIPO', 'DOCUMENTO', 'DATA',
    'PT', 'FONTE', 'NATUREZA', 'C. CONTÁBIL', 'VALOR'])  # Cabeçalho da tabela
# table_data.extend(dataframe.values.tolist())  # Dados do DataFrame

# for i in table_data: print(i)